In [1]:
%reload_ext autoreload
%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [2]:
from src.train import train
from src.augment import ImageBatchProcessor
from src.crop import crop_images_from_folder
from src.model import model
from src.dataset import dataset
seed = 42
image_folder = 'dataset2/train/images/'
label_folder = 'dataset2/train/labels/'
target_image_folder = 'dataset2/train/images_aug/'
target_label_folder = 'dataset2/train/labels_aug/'

In [ ]:
processor = ImageBatchProcessor(image_folder, label_folder, target_image_folder, target_label_folder)
processor.augment_and_save(nb_augmentation=3)

In [ ]:
crop_images_from_folder(target_image_folder, target_label_folder, 'dataset2/train/aug_cropped_images/')
crop_images_from_folder(image_folder, label_folder, 'dataset2/train/cropped_images/')
val_path = 'dataset2/val/images/'
val_label_path = 'dataset2/val/labels/'
crop_images_from_folder(val_path, val_label_path, 'dataset2/val/cropped_images/')

In [3]:
val_path = 'dataset2/val/cropped_images/'
aug_train_path = 'dataset2/train/aug_cropped_images/'
train_path = 'dataset2/train/cropped_images/'
standard_size = (32, 32)
label_size_factor = -1
train_data = dataset(img_dir=train_path, augment_path=aug_train_path, label_size_factor=label_size_factor, standard_size=standard_size)
val_data = dataset(val_path, train=False, standard_size=standard_size)


Augmenting label ceder with -498 images
Augmenting label interdiction with -730 images
Augmenting label frouge with -455 images
Augmenting label fvert with -474 images
Augmenting label obligation with -473 images
Augmenting label none with -412 images
Augmenting label danger with -527 images
Augmenting label forange with -432 images
Augmenting label stop with -465 images


Preprocessing images: 100%|██████████| 1181/1181 [00:11<00:00, 99.22it/s] 


Total loading time: 16.39 seconds


Preprocessing images: 100%|██████████| 135/135 [00:01<00:00, 113.82it/s]

Total loading time: 1.62 seconds


In [4]:
label_count = {}
for x in train_data.images:
    label = x.label
    if label in label_count.keys():
        label_count[label] +=1
    else:
        label_count[label] =1
label_count   

{'ceder': 133,
 'interdiction': 365,
 'frouge': 90,
 'fvert': 109,
 'obligation': 108,
 'none': 47,
 'danger': 162,
 'forange': 67,
 'stop': 100}

In [ ]:
max_iter = 10000
# initialize the model
my_model = model(seed)
#my_model.train_lr(train_data, verbose=1, max_iter=max_iter)
#my_model.grid_search_svm(train_data, verbose=2, max_iter=max_iter)
my_model.train_svm(train_data, verbose=1, max_iter=max_iter)
#my_model.train_OneVsRest(train_data, verbose=1, max_iter=max_iter)
#my_model.plot_learning_curve(train_data, verbose=1)
print(f"Model trained with {my_model.name} with seed {seed},max_iter {max_iter}, standard_size {standard_size}, label_size_factor {label_size_factor}")
# evaluate the model
accuracy = my_model.evaluate(val_data)
print("Model accuracy with Logistic Regression: {}".format(accuracy))

In [ ]:
#use model.predict_window(self, window) to predict a window
